In [ ]:
import librosa
import matplotlib as mpl
import numpy as np
import random
import rosbag
import struct
import tams_pr2_guzheng
import tams_pr2_guzheng.utils as utils
import pandas as pd
import seaborn as sns

from collections import Counter

from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib.animation import FuncAnimation
from tams_pr2_guzheng.paths import RuckigPath
from IPython import display

mpl.style.use('seaborn-v0_8')

%load_ext autoreload
%autoreload 1
import sys
if '../utils' not in sys.path:
    sys.path.append('../utils')
import u
%aimport u

In [ ]:
D= pd.read_json('../../../../data/20230823 - first active explorations/plucks_explore_fis4.json')
#D= pd.read_json('../../../../data/20230823 - first active explorations/plucks_explore_d4_2.json')
D.columns

In [ ]:
DF= D[['string','keypoint_pos_y', 'string_position', 'detected_note', 'loudness', 'unexpected_onsets', 'min_distance']].copy()
DF.describe()

In [ ]:
string_len = DF['string_position'].max()
actionspace = RuckigPath.ActionSpace(
    string_position= np.array((0.0, string_len)),
    keypoint_pos_y= np.array((-0.007, 0.015)),
    keypoint_pos_z= np.array((-0.004,)),
    keypoint_vel_y= np.array((0.015,)),
    keypoint_vel_z= np.array((0.015,)),
)

features_norm_params = (
    np.array([actionspace.string_position[1]/2, (actionspace.keypoint_pos_y[0] + (actionspace.keypoint_pos_y[1]-actionspace.keypoint_pos_y[0])/2) ]),
    np.array([actionspace.string_position[1]/4, (actionspace.keypoint_pos_y[1] - actionspace.keypoint_pos_y[0])/4])
)

DG = utils.make_grid_points(actionspace, 50)

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(18,5))
cmap= sns.cubehelix_palette(as_cmap=True)

DL = DF[DF['loudness'].notnull()].copy()

features = DL[['string_position', 'keypoint_pos_y']]
value = DL['loudness'] #.fillna(0.0)
utils.plot_trials(DF, DF['loudness'], ax= ax[0], actionspace= actionspace)

gp_loudness = utils.fit_gp(utils.normalize(features, features_norm_params), value, normalize= True, alpha= 1.0, rbf_length= 1.5)
GridPred= pd.DataFrame()
GridPred['mean'], GridPred['std'] = gp_loudness.predict(utils.normalize(DG.values, features_norm_params), return_std= True)
DL['safe_score'] = utils.score_safety(DF)

utils.grid_plot(GridPred['mean'], actionspace, cmap= cmap, ax= ax[1])
ax[1].set_title('loudness mean prediction')
utils.grid_plot(GridPred['std'], actionspace, cmap= cmap, ax= ax[2])
ax[2].set_title('loudness std prediction');

In [ ]:
Grid = DG.merge(GridPred, left_index= True, right_index= True)
keypoint_pos_y = (Grid['keypoint_pos_y'].unique()[0])
Gr = Grid[Grid['keypoint_pos_y'] == keypoint_pos_y]
sns.lineplot(data=Gr, x='string_position', y= 'mean', estimator= None)
plt.fill_between(Gr['string_position'], Gr['mean']-Gr['std'], Gr['mean']+Gr['std'], alpha= 0.5)

sns.scatterplot(data= DF[np.abs(DF['keypoint_pos_y']-keypoint_pos_y) < 0.001], x='string_position', y= 'loudness')

In [ ]:

# create mpl animation over all keypoint_pos_y
keypoint_pos_y = Grid['keypoint_pos_y'].unique()
fig, ax = plt.subplots()

def animate(i):
    ax.clear()
    ax.set_title(f'keypoint_pos_y = {keypoint_pos_y[i]:.4f}')
    Gr = Grid[Grid['keypoint_pos_y'] == keypoint_pos_y[i]]
    sns.lineplot(data=Gr, x='string_position', y= 'mean', estimator= None, ax= ax)
    ax.fill_between(Gr['string_position'], Gr['mean']-Gr['std'], Gr['mean']+Gr['std'], alpha= 0.5)
    sns.scatterplot(data= DF[np.abs(DF['keypoint_pos_y']-keypoint_pos_y[i]) < 0.001], x='string_position', y= 'loudness', ax= ax)

    # ax.set_ylim(-0.5, 1.5)
    # ax.set_xlim(0, 0.5)
    # ax.set_xlabel('string_position')
    # ax.set_ylabel('loudness')
    return ax

anim = FuncAnimation(fig, animate, frames= len(keypoint_pos_y), interval= 300)
display.HTML(anim.to_html5_video())
#plt.close()

In [ ]:
# create mpl animation over all keypoint_pos_y
string_position = Grid['string_position'].unique()
fig, ax = plt.subplots()

def animate_sp(i):
    ax.clear()
    ax.set_title(f'string_position = {string_position[i]:.4f}')
    Gr = Grid[Grid['string_position'] == string_position[i]]
    sns.lineplot(data=Gr, x='keypoint_pos_y', y= 'mean', estimator= None, ax= ax)
    ax.fill_between(Gr['keypoint_pos_y'], Gr['mean']-Gr['std'], Gr['mean']+Gr['std'], alpha= 0.5)
    sns.scatterplot(data= DF[np.abs(DF['string_position']-string_position[i]) < 0.015], x='keypoint_pos_y', y= 'loudness', ax= ax)

    #ax.set_ylim(-0.5, 1.5)
    ax.set_xlim(Grid['keypoint_pos_y'].min(), Grid['keypoint_pos_y'].max())
    ax.set_ylim(DF['loudness'].min(), DF['loudness'].max())
    ax.set_xlabel('keypoint_pos_y')
    ax.set_ylabel('loudness')
    return ax

anim = FuncAnimation(fig, animate_sp, frames= len(string_position), interval= 200)
display.HTML(anim.to_html5_video())
#plt.close()